# ClipCard Field Evaluation: Clinical/SBAR
**Purpose:** This notebook analyzes historical data from the `clipmap` CSVs to evaluate the real-world performance of the ClipCard system over time for the **Clinical/SBAR** silo.
It calculates and visualizes key metrics as defined in the `dfm-clinical-sbar.md` framework document, such as:- Near Miss Rate- Preventable Escalations- Recheck On-Time %

In [ ]:
import pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport matplotlib.dates as mdates# Load the aggregated ClipCard datatry:    cards_df = pd.read_csv('../../data/clipmap/cards.csv')    events_df = pd.read_csv('../../data/clipmap/events.csv')    # Filter for the correct domain and merge    clinical_cards = cards_df[cards_df['domain'] == 'dfm-clinical-sbar'].copy()    merged_df = pd.merge(clinical_cards, events_df, left_on='id', right_on='card_id')    merged_df['time'] = pd.to_datetime(merged_df['time'])except FileNotFoundError as e:    print(f'ERROR: {e.filename} not found. Please ensure the data exists.')    merged_df = pd.DataFrame()

### Metric 1: Near Miss Rate
**Definition:** Near-miss events logged per 100 SBAR handoffs for high-risk cases.**Goal:** Increase vs. baseline (better detection).
**Note:** This metric requires data from safety reporting systems (e.g., RL Solutions) or specific 'near miss' event types, which are not in the current `events.csv`. We will create a placeholder visualization with synthetic data.

In [ ]:
if not merged_df.empty:    # Group by week for trend analysis    events_by_week = merged_df.set_index('time').resample('W')        # --- Placeholder Data Generation ---     # In a real scenario, this would come from AAR logs or specific event types.    placeholder_df = events_by_week.size().to_frame(name='total_handoffs_proxy')    # Assume a near-miss reporting rate that is increasing over time due to better culture    placeholder_df['near_misses'] = np.random.randint(1, 3, size=len(placeholder_df)) + np.arange(len(placeholder_df))    placeholder_df['near_miss_rate'] = (placeholder_df['near_misses'] / placeholder_df['total_handoffs_proxy'] * 100).fillna(0)    # --- End Placeholder ---     # Plotting    fig, ax = plt.subplots(figsize=(12, 6))    ax.plot(placeholder_df.index, placeholder_df['near_miss_rate'], marker='o', linestyle='-', label='Near Miss Rate (per 100 handoffs)')    ax.set_title('Near Miss Reporting Rate Over Time (Synthetic Data)')    ax.set_ylabel('Rate (per 100 handoffs)')    ax.set_xlabel('Week')    ax.grid(True, which='both', linestyle='--', linewidth=0.5)    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))    plt.xticks(rotation=45)    ax.legend()    plt.tight_layout()    plt.show()else:    print('Data is empty. Cannot calculate Near Miss Rate.')

### Metric 2: Preventable Escalations
**Definition:** Escalations judged preventable during Root Cause Analysis (RCA) per 100 high-risk interventions.**Goal:** Decrease vs. baseline.
**Note:** This also requires RCA data. We will use 'Incident Prevented' as a proxy for an 'escalation' and generate synthetic data for the 'preventable' judgment.

In [ ]:
if not merged_df.empty:    escalations_df = merged_df[merged_df['outcome'] == 'Incident Prevented'].copy()    escalations_by_week = escalations_df.set_index('time').resample('W').size().to_frame(name='escalations')        # --- Placeholder Data Generation ---     # Assume the proportion of escalations judged 'preventable' decreases over time    preventable_proportion = 0.6 - (np.arange(len(escalations_by_week)) * 0.15)    preventable_proportion[preventable_proportion < 0.1] = 0.1 # Floor at 10%    escalations_by_week['preventable_escalations'] = (escalations_by_week['escalations'] * preventable_proportion).astype(int)    escalations_by_week['preventable_rate'] = (escalations_by_week['preventable_escalations'] / escalations_by_week['escalations'] * 100).fillna(0)    # --- End Placeholder ---         # Plotting    fig, ax = plt.subplots(figsize=(12, 6))    ax.plot(escalations_by_week.index, escalations_by_week['preventable_rate'], marker='s', linestyle='--', color='orange', label='Preventable Escalation Rate (%)')    ax.set_title('Rate of Preventable Escalations Over Time (Synthetic Data)')    ax.set_ylabel('Preventable Rate (%)')    ax.set_xlabel('Week')    ax.grid(True, which='both', linestyle='--', linewidth=0.5)    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))    plt.xticks(rotation=45)    ax.legend()    plt.tight_layout()    plt.show()else:    print('Data is empty. Cannot calculate Preventable Escalations.')